In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
csv = 'twitter_data_2.csv'
my_df = pd.read_csv(csv)
my_df.head()

,clean_text,category
0,when modi promised â€œminimum government maxim...,-1
1,talk all the nonsense and continue all the dra...,0
2,what did just say vote for modi welcome bjp t...,1
3,asking his supporters prefix chowkidar their n...,1
4,answer who among these the most powerful world...,1


In [3]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49998 entries, 0 to 49997
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clean_text  49998 non-null  object
 1   category    49998 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.3+ KB


In [4]:
x = my_df.clean_text
y = my_df.category

In [5]:
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [6]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [7]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [8]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [9]:
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])


100%|██████████| 49998/49998 [00:00<00:00, 2380894.55it/s]


In [10]:
%%time
for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha


100%|██████████| 49998/49998 [00:00<00:00, 2509955.85it/s]


CPU times: user 58.1 s, sys: 328 ms, total: 58.4 s
Wall time: 31.3 s


In [11]:
model_ug_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 49998/49998 [00:00<00:00, 1647809.37it/s]


In [12]:
%%time
for epoch in range(30):
    model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_sg.alpha -= 0.002
    model_ug_sg.min_alpha = model_ug_sg.alpha


100%|██████████| 49998/49998 [00:00<00:00, 2397992.15it/s]


CPU times: user 1min 53s, sys: 440 ms, total: 1min 53s
Wall time: 59.1 s


In [13]:
model_ug_cbow.save('w2v_model_ug_cbow.word2vec')
model_ug_sg.save('w2v_model_ug_sg.word2vec')

In [14]:
#prepare for CNN
from gensim.models import KeyedVectors
model_ug_cbow = KeyedVectors.load('w2v_model_ug_cbow.word2vec')
model_ug_sg = KeyedVectors.load('w2v_model_ug_sg.word2vec')


In [15]:
len(model_ug_cbow.wv.vocab.keys())

21638

In [16]:
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])
print('Found %s word vectors.' % len(embeddings_index))


Found 21638 word vectors.


In [17]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)


In [18]:
len(tokenizer.word_index)

51836

In [19]:
for x in x_train[:5]:
    print(x)

nationalist think why should stand behind modiji
tourism receives major boost under modi government

kunal kamra pretentious chutiya modi did more work the last five years than the braincells you have lost since your birth while smoking weed
top cabinet ministers including national security advisor residence his announcement related security stay tuned 
its was failure govt that why modi govt gave free hands army and decide the strike
this mentality why progressives totally unhinged when they see narendra modi become prime minister after starting out streetside tea vendor does not compute 


In [20]:
sequences[:5]


[[1226, 93, 33, 56, 492, 547, 414, 2727, 8602, 815, 2019, 83, 1, 76],
 [6774,
  5448,
  15453,
  1889,
  1,
  61,
  47,
  102,
  2,
  123,
  526,
  53,
  69,
  2,
  21375,
  5,
  14,
  370,
  203,
  26,
  2230,
  232,
  4782,
  5197],
 [509, 976, 1023, 726, 192, 371, 5198, 9597, 19, 208, 1227, 371, 847, 4430],
 [36, 20, 542, 35, 11, 33, 1, 35, 244, 237, 727, 487, 3, 1092, 2, 483],
 [7,
  2394,
  33,
  12580,
  830,
  21376,
  46,
  21,
  91,
  31,
  1,
  147,
  103,
  82,
  59,
  2164,
  64,
  21377,
  1292,
  7271,
  165,
  8,
  12581]]

In [21]:
length = []
for x in x_train:
    length.append(len(x.split()))

In [22]:
max(length)


49

In [23]:
x_train_seq = pad_sequences(sequences, maxlen=45)
print('Shape of data tensor:', x_train_seq.shape)


Shape of data tensor: (48998, 45)


In [24]:
x_train_seq[:5]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,  1226,    93,    33,    56,   492,
          547,   414,  2727,  8602,   815,  2019,    83,     1,    76],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,  6774,  5448, 15453,  1889,     1,    61,
           47,   102,     2,   123,   526,    53,    69,     2, 21375,
            5,    14,   370,   203,    26,  2230,   232,  4782,  5197],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,   509,   976,  1023,   726,   192,
    

In [25]:
sequences_val = tokenizer.texts_to_sequences(x_validation)
x_val_seq = pad_sequences(sequences_val, maxlen=45)

In [26]:
num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [27]:
np.array_equal(embedding_matrix[6] ,embeddings_index.get('you'))


False

In [28]:
seed = 7

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [29]:
model_ptw2v = Sequential()
e = Embedding(100000, 200, weights=[embedding_matrix], input_length=45, trainable=False)
model_ptw2v.add(e)
model_ptw2v.add(Flatten())
model_ptw2v.add(Dense(256, activation='relu'))
model_ptw2v.add(Dense(1, activation='sigmoid'))
model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_ptw2v.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=16, verbose=2)


Epoch 1/5
3063/3063 - 18s - loss: -8.5894e-01 - accuracy: 0.4424 - val_loss: -4.0787e+00 - val_accuracy: 0.4480
Epoch 2/5
3063/3063 - 6s - loss: -6.4474e+01 - accuracy: 0.4737 - val_loss: -9.2044e+01 - val_accuracy: 0.4600
Epoch 3/5
3063/3063 - 6s - loss: -3.5132e+02 - accuracy: 0.4926 - val_loss: -2.9840e+02 - val_accuracy: 0.4800
Epoch 4/5
3063/3063 - 6s - loss: -8.9109e+02 - accuracy: 0.5067 - val_loss: -6.1698e+02 - val_accuracy: 0.4760
Epoch 5/5
3063/3063 - 6s - loss: -1.6820e+03 - accuracy: 0.5138 - val_loss: -9.8182e+02 - val_accuracy: 0.4720


In [30]:
from keras.layers import Conv1D, GlobalMaxPooling1D

In [31]:
structure_test = Sequential()
e = Embedding(100000, 200, input_length=45)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 45, 200)           20000000  
_________________________________________________________________
conv1d (Conv1D)              (None, 44, 100)           40100     
Total params: 20,040,100
Trainable params: 20,040,100
Non-trainable params: 0
_________________________________________________________________


In [32]:
structure_test = Sequential()
e = Embedding(100000, 200, input_length=45)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 45, 200)           20000000  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 44, 100)           40100     
Total params: 20,040,100
Trainable params: 20,040,100
Non-trainable params: 0
_________________________________________________________________


In [33]:
structure_test = Sequential()
e = Embedding(100000, 200, input_length=45)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.add(GlobalMaxPooling1D())
structure_test.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 45, 200)           20000000  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 44, 100)           40100     
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
Total params: 20,040,100
Trainable params: 20,040,100
Non-trainable params: 0
_________________________________________________________________


In [34]:
#static word vectors extracted from Word2Vec
model_cnn_01 = Sequential()
e = Embedding(100000, 200, weights=[embedding_matrix], input_length=45, trainable=False)
model_cnn_01.add(e)
model_cnn_01.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_01.add(GlobalMaxPooling1D())
model_cnn_01.add(Dense(256, activation='relu'))
model_cnn_01.add(Dense(1, activation='sigmoid'))
model_cnn_01.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=10, batch_size=16, verbose=2)

Epoch 1/10
3063/3063 - 10s - loss: -3.0523e+03 - accuracy: 0.5026 - val_loss: -1.8120e+04 - val_accuracy: 0.5460
Epoch 2/10
3063/3063 - 7s - loss: -5.0435e+04 - accuracy: 0.5224 - val_loss: -1.3863e+05 - val_accuracy: 0.5460
Epoch 3/10
3063/3063 - 7s - loss: -1.9720e+05 - accuracy: 0.5235 - val_loss: -4.1150e+05 - val_accuracy: 0.5440
Epoch 4/10
3063/3063 - 7s - loss: -4.7428e+05 - accuracy: 0.5256 - val_loss: -8.8411e+05 - val_accuracy: 0.5300
Epoch 5/10
3063/3063 - 7s - loss: -9.1683e+05 - accuracy: 0.5246 - val_loss: -1.5918e+06 - val_accuracy: 0.5360
Epoch 6/10
3063/3063 - 7s - loss: -1.5649e+06 - accuracy: 0.5251 - val_loss: -2.5863e+06 - val_accuracy: 0.5420
Epoch 7/10
3063/3063 - 7s - loss: -2.4492e+06 - accuracy: 0.5250 - val_loss: -3.8369e+06 - val_accuracy: 0.5480
Epoch 8/10
3063/3063 - 7s - loss: -3.5977e+06 - accuracy: 0.5263 - val_loss: -5.5765e+06 - val_accuracy: 0.5440
Epoch 9/10
3063/3063 - 7s - loss: -5.0482e+06 - accuracy: 0.5269 - val_loss: -7.7152e+06 - val_accuracy

In [35]:
#word embedding being learned from scratch with embedding layer
model_cnn_02 = Sequential()
e = Embedding(100000, 200, input_length=45)
model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dense(1, activation='sigmoid'))
model_cnn_02.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_02.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)


Epoch 1/5
1532/1532 - 269s - loss: -3.4739e+05 - accuracy: 0.5655 - val_loss: -2.3189e+06 - val_accuracy: 0.5880
Epoch 2/5
1532/1532 - 267s - loss: -9.5566e+06 - accuracy: 0.6019 - val_loss: -2.9286e+07 - val_accuracy: 0.5920
Epoch 3/5
1532/1532 - 267s - loss: -5.1903e+07 - accuracy: 0.6052 - val_loss: -1.1423e+08 - val_accuracy: 0.5960
Epoch 4/5
1532/1532 - 267s - loss: -1.5317e+08 - accuracy: 0.6066 - val_loss: -2.9047e+08 - val_accuracy: 0.6040
Epoch 5/5
1532/1532 - 266s - loss: -3.4235e+08 - accuracy: 0.6081 - val_loss: -5.9345e+08 - val_accuracy: 0.6040


In [37]:
#Word2Vec word vectors being updated through training.
model_cnn_03 = Sequential()
e = Embedding(100000, 200, weights=[embedding_matrix], input_length=45, trainable=True)
model_cnn_03.add(e)
model_cnn_03.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_03.add(GlobalMaxPooling1D())
model_cnn_03.add(Dense(256, activation='relu'))
model_cnn_03.add(Dense(1, activation='sigmoid'))
model_cnn_03.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_03.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=4, batch_size=32, verbose=2)

Epoch 1/4
1532/1532 - 262s - loss: -1.3077e+05 - accuracy: 0.5660 - val_loss: -9.8854e+05 - val_accuracy: 0.6080
Epoch 2/4
1532/1532 - 260s - loss: -5.1990e+06 - accuracy: 0.6297 - val_loss: -1.6420e+07 - val_accuracy: 0.6180
Epoch 3/4
1532/1532 - 260s - loss: -3.1344e+07 - accuracy: 0.6373 - val_loss: -6.9443e+07 - val_accuracy: 0.6300
Epoch 4/4


KeyboardInterrupt: ignored

In [38]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

tweet_input = Input(shape=(45,), dtype='int32')

tweet_encoder = Embedding(100000, 200, weights=[embedding_matrix], input_length=45, trainable=True)(tweet_input)
bigram_branch = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.2)(merged)
merged = Dense(1)(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 45)]         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 45, 200)      20000000    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 44, 100)      40100       embedding_8[0][0]                
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 43, 100)      60100       embedding_8[0][0]                
______________________________________________________________________________________________

In [40]:
from keras.callbacks import ModelCheckpoint

filepath="CNN_best_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.fit(x_train_seq, y_train, batch_size=32, epochs=5,
                     validation_data=(x_val_seq, y_validation), callbacks = [checkpoint])


Epoch 1/5
1532/1532 [==============================] - 267s 174ms/step - loss: -13380058.0000 - accuracy: 0.6189 - val_loss: -42486248.0000 - val_accuracy: 0.6060


Epoch 2/5
1532/1532 [==============================] - 267s 174ms/step - loss: -80472448.0000 - accuracy: 0.6249 - val_loss: -177928352.0000 - val_accuracy: 0.6100


Epoch 3/5
1532/1532 [==============================] - 265s 173ms/step - loss: -250034208.0000 - accuracy: 0.6297 - val_loss: -472905088.0000 - val_accuracy: 0.6080


Epoch 4/5
1532/1532 [==============================] - 265s 173ms/step - loss: -581103872.0000 - accuracy: 0.6295 - val_loss: -997949632.0000 - val_accuracy: 0.6080


Epoch 5/5
1532/1532 [==============================] - 265s 173ms/step - loss: -1124566784.0000 - accuracy: 0.6301 - val_loss: -1819132288.0000 - val_accuracy: 0.6120


In [ ]:
from keras.models import load_model
loaded_CNN_model = load_model('CNN_best_weights.02-0.8333.hdf5')
loaded_CNN_model.evaluate(x=x_val_seq, y=y_validation)

In [ ]:
#final model evaluation
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(max_features=100000,ngram_range=(1, 3))
tvec.fit(x_train)


In [ ]:
x_train_tfidf = tvec.transform(x_train)
x_test_tfidf = tvec.transform(x_test)

In [ ]:
lr_with_tfidf = LogisticRegression()
lr_with_tfidf.fit(x_train_tfidf,y_train)

In [ ]:
lr_with_tfidf.score(x_test_tfidf,y_test)

In [ ]:
yhat_lr = lr_with_tfidf.predict_proba(x_test_tfidf)

In [ ]:
sequences_test = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(sequences_test, maxlen=45)

In [ ]:
loaded_CNN_model.evaluate(x=x_test_seq, y=y_test)

In [ ]:
yhat_cnn = loaded_CNN_model.predict(x_test_seq)

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, threshold = roc_curve(y_test, yhat_lr[:,1])
roc_auc = auc(fpr, tpr)
fpr_cnn, tpr_cnn, threshold = roc_curve(y_test, yhat_cnn)
roc_auc_nn = auc(fpr_cnn, tpr_cnn)
plt.figure(figsize=(8,7))
plt.plot(fpr, tpr, label='tfidf-logit (area = %0.3f)' % roc_auc, linewidth=2)
plt.plot(fpr_cnn, tpr_cnn, label='w2v-CNN (area = %0.3f)' % roc_auc_nn, linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', linewidth=2)
plt.xlim([-0.05, 1.0])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate', fontsize=18)
plt.ylabel('True Positive Rate', fontsize=18)
plt.title('Receiver operating characteristic: is positive', fontsize=18)
plt.legend(loc="lower right")
plt.show()

In [ ]:
from google.colab import files
files.download('w2v_model_ug_cbow.word2vec')
files.download('w2v_model_ug_sg.word2vec')
